# Definition

In [1]:
forward_seq = 4
backward_seq = 4
valid_tropical_seq_num =4
predict_seq_num = 4
trainYear = (2000, 2014)
testYear = (2015, 2018)

In [2]:
"""
typhoonYear
tid Typhoon ID
typhoonRecordNum 
typhoonName 
typhoonRecords 
"""
class TyphoonHeader:

    def __init__(self, typhoonYear, tid):
        self.typhoonYear = typhoonYear
        self.tid = tid
        self.typhoonRecordNum = 0
        self.typhoonRecords = []

    def printer(self):
        print("tid: %d, typhoonYear: %d, typhoonYear: %s, typhoonRecordNum: %d" %
              (self.tid, self.typhoonYear, self.typhoonName, self.typhoonRecordNum))
        for typhoonRecord in self.typhoonRecords:
            typhoonRecord.printer()

In [3]:
# 台风记录信息类
"""
typhoonTime Tyhoon Record Time
lat 
long 
wind 
totalNum Typhoon Record ID
"""
class TyphoonRecord:

    def __init__(self, typhoonTime, lat, long, wind, totalNum):
        self.typhoonTime = typhoonTime
        self.lat = lat
        self.long = long
        self.wind = wind
        self.totalNum = totalNum

    def printer(self):
        print("totalNum: %d, typhoonTime: %d, lat: %d, long: %d, wind: %d" %
              (self.totalNum, self.typhoonTime, self.lat, self.long, self.wind))


In [4]:
import math
import numpy as np

def get_vector_angle(vector1, vector2):
    a_abs = math.sqrt(vector1[0]*vector1[0] + vector1[1] * vector1[1])
    b_abs = math.sqrt(vector2[0]*vector2[0] + vector2[1] * vector2[1])
    a_b = vector1[0]*vector2[0] + vector1[1]*vector2[1]
    cos_angle = a_b/(a_abs*b_abs)
    if cos_angle > 1:
        cos_angle = 1
    elif cos_angle < -1:
        cos_angle = -1
    angle = np.arccos(cos_angle) * 360 / 2 / np.pi
    return angle

In [5]:
def read_cma_dfs(cma_list):
    totalNum = 1
    typhoonHeaderList = []
    for df in cma_list:
        tid = df.loc[0, 'TID']
        typhoonYear = df.loc[0, 'YEAR']
        typhoonHeader = TyphoonHeader(typhoonYear, tid, )
        for i in range(len(df)):
            typhoonTime = int(
                str(df.loc[i, 'YEAR'])+
                str(df.loc[i, 'MONTH']).zfill(2)+
                str(df.loc[i, 'DAY']).zfill(2)+
                str(df.loc[i, 'HOUR']).zfill(2)
            )
            lat = df.loc[i, 'LAT'] * 0.1
            long = df.loc[i, 'LONG'] * 0.1
            wind = df.loc[i, 'WND']
            typhoonRecord = TyphoonRecord(typhoonTime, lat, long, wind, totalNum)
            totalNum += 1
            typhoonHeader.typhoonRecords.append(typhoonRecord)
        typhoonHeader.typhoonRecordNum = len(typhoonHeader.typhoonRecords)
        typhoonHeaderList.append(typhoonHeader)
    return typhoonHeaderList

In [6]:
from geopy.distance import great_circle

def buildup_feature(typhoonRecordsList, tid, fileXWriter, pred_num):
    for start in range(forward_seq, len(typhoonRecordsList) - predict_seq_num):
        strXLine = str(typhoonRecordsList[start].totalNum) + \
                   ',' + str(tid) + \
                   ',' + str(typhoonRecordsList[start].typhoonTime) + \
                   ',' + str(typhoonRecordsList[start + pred_num].lat) + \
                   ',' + str(typhoonRecordsList[start + pred_num].long) + \
                   ',' + str(typhoonRecordsList[start].typhoonTime//1000000) + \
                   ',' + str(typhoonRecordsList[start].lat) + \
                   ',' + str(typhoonRecordsList[start].long) + \
                   ',' + str(typhoonRecordsList[start].wind)

        # 前24小时的数据纬度
        strXLine += ',' + str(typhoonRecordsList[start - 1].lat)
        strXLine += ',' + str(typhoonRecordsList[start - 2].lat)
        strXLine += ',' + str(typhoonRecordsList[start - 3].lat)
        strXLine += ',' + str(typhoonRecordsList[start - 4].lat)
        
        # 前24小时的数据纬度
        strXLine += ',' + str(typhoonRecordsList[start - 1].long)
        strXLine += ',' + str(typhoonRecordsList[start - 2].long)
        strXLine += ',' + str(typhoonRecordsList[start - 3].long)
        strXLine += ',' + str(typhoonRecordsList[start - 4].long)
        
        # 前24小时的数据风速
        strXLine += ',' + str(typhoonRecordsList[start - 1].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 2].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 3].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 4].wind)

        # 增加月份
        strXLine += ',' + str(typhoonRecordsList[start].typhoonTime//10000 % 100)
        
        # 风速差
        strXLine += ',' + str(typhoonRecordsList[start].wind - typhoonRecordsList[start - 1].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 1].wind - typhoonRecordsList[start - 2].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 2].wind - typhoonRecordsList[start - 3].wind)
        strXLine += ',' + str(typhoonRecordsList[start - 3].wind - typhoonRecordsList[start - 4].wind)

        # 纬度差 0-6, 0-12,0-18,0-24
        latdiff = []
        latdiff.append((typhoonRecordsList[start].lat - typhoonRecordsList[start - 1].lat))
        strXLine += ',' + str(latdiff[-1])
        latdiff.append((typhoonRecordsList[start - 1].lat - typhoonRecordsList[start - 2].lat))
        strXLine += ',' + str(latdiff[-1])
        latdiff.append((typhoonRecordsList[start - 2].lat - typhoonRecordsList[start - 3].lat))
        strXLine += ',' + str(latdiff[-1])
        latdiff.append((typhoonRecordsList[start - 3].lat - typhoonRecordsList[start - 4].lat))
        strXLine += ',' + str(latdiff[-1])

        # 经度差
        longdiff = []
        longdiff.append((typhoonRecordsList[start].long - typhoonRecordsList[start - 1].long))
        strXLine += ',' + str(longdiff[-1])
        longdiff.append((typhoonRecordsList[start - 1].long - typhoonRecordsList[start - 2].long))
        strXLine += ',' + str(longdiff[-1])
        longdiff.append((typhoonRecordsList[start - 2].long - typhoonRecordsList[start - 3].long))
        strXLine += ',' + str(longdiff[-1])
        longdiff.append((typhoonRecordsList[start - 3].long - typhoonRecordsList[start - 4].long))
        strXLine += ',' + str(longdiff[-1])

        # 纬度差的平方和开根号
        sum = 0
        for i in range(len(latdiff)):
            sum += latdiff[i]**2
        strXLine += ',' + str(sum)
        strXLine += ',' + str("{:.4f}".format(math.sqrt(sum)))

        # 经度差的平方和开根号
        sum = 0
        for i in range(len(latdiff)):
            sum += longdiff[i] ** 2
        strXLine += ',' + str(sum)
        strXLine += ',' + str("{:.4f}".format(math.sqrt(sum)))

        # 物理加速度
        strXLine += ',' + str("{:.4f}".format(
            2 * great_circle(
                (typhoonRecordsList[start].lat, typhoonRecordsList[start].long),
                (typhoonRecordsList[start - 1].lat,typhoonRecordsList[start - 1].long)
            ).kilometers / (6 ** 2)
        ))
        strXLine += ',' + str("{:.4f}".format(
            2 * great_circle(
                (typhoonRecordsList[start - 1].lat,typhoonRecordsList[start - 1].long),
                (typhoonRecordsList[start - 2].lat,typhoonRecordsList[start - 2].long)
            ).kilometers / (6 ** 2)
        ))
        strXLine += ',' + str("{:.4f}".format(
            2 * great_circle(
                (typhoonRecordsList[start].lat,typhoonRecordsList[start].long),
                (typhoonRecordsList[start - 2].lat,typhoonRecordsList[start - 2].long)
            ).kilometers / (12 ** 2)
                                            ))
        strXLine += ',' + str("{:.4f}".format(
            2 * great_circle(
                (typhoonRecordsList[start -2].lat,typhoonRecordsList[start -2].long),
                (typhoonRecordsList[start - 4].lat,typhoonRecordsList[start - 4].long)
            ).kilometers / (12 ** 2)
        ))

        # 当前纬度开根号
        strXLine += ',' + str("{:.4f}".format(math.sqrt(typhoonRecordsList[start].lat)))

        # 当前经度开根号
        strXLine += ',' + str("{:.4f}".format(math.sqrt(typhoonRecordsList[start].long)))

        ########################################################################################################

        # 纬向加速度
        strXLine += ',' + str(
            (typhoonRecordsList[start].lat - typhoonRecordsList[start - 1].lat) -
            (typhoonRecordsList[start - 1].lat - typhoonRecordsList[start - 2].lat)
        )

        strXLine += ',' + str(
            (typhoonRecordsList[start - 2].lat - typhoonRecordsList[start - 3].lat) -
            (typhoonRecordsList[start - 3].lat - typhoonRecordsList[start - 4].lat)
        )
        
        # 经向加速度
        strXLine += ',' + str(
            (typhoonRecordsList[start].long - typhoonRecordsList[start - 1].long) -
            (typhoonRecordsList[start - 1].long - typhoonRecordsList[start - 2].long)
        )

        strXLine += ',' + str(
            (typhoonRecordsList[start - 2].long - typhoonRecordsList[start - 3].long) -
            (typhoonRecordsList[start - 3].long - typhoonRecordsList[start - 4].long)
        )
        
        # 计算纬度夹角
        for i in range(1, forward_seq + 1):
            diff_lat = typhoonRecordsList[start - i + 1].lat - typhoonRecordsList[start - i].lat
            diff_long = typhoonRecordsList[start - i + 1].long - typhoonRecordsList[start - i].long
            vector1 = [diff_lat, diff_long]
            vector2 = [1, 0]
            if diff_lat < 0 and diff_long < 0:
                strXLine += ',' + str(90 + get_vector_angle(vector1, vector2))
            elif diff_lat > 0 and diff_long < 0:
                strXLine += ',' + str(270 + get_vector_angle(vector1, vector2))
            elif diff_lat == 0 and diff_long == 0:
                strXLine += ',' + str(0)
            else:
                strXLine += ',' + str(get_vector_angle(vector1, vector2))

        # 计算经度度夹角
        for i in range(1, forward_seq + 1):
            diff_lat = typhoonRecordsList[start - i + 1].lat - typhoonRecordsList[start - i].lat
            diff_long = typhoonRecordsList[start - i + 1].long - typhoonRecordsList[start - i].long
            vector1 = [diff_lat, diff_long]
            vector2 = [0, 1]
            if diff_lat > 0 and diff_long < 0:
                strXLine += ',' + str(90 + get_vector_angle(vector1, vector2))
            elif diff_lat > 0 and diff_long > 0:
                strXLine += ',' + str(270 + get_vector_angle(vector1, vector2))
            elif diff_lat == 0 and diff_long == 0:
                strXLine += ',' + str(0)
            else:
                strXLine += ',' + str(get_vector_angle(vector1, vector2))
        
        # 计算热带气旋两个路径的夹角
        for i in range(1, forward_seq):
            diff_lat1 = typhoonRecordsList[start - i + 1].lat - typhoonRecordsList[start - i].lat
            diff_long1 = typhoonRecordsList[start - i + 1].long - typhoonRecordsList[start - i].long
            vector1 = [diff_lat1, diff_long1]
            diff_lat2 = typhoonRecordsList[start - i - 1].lat - typhoonRecordsList[start - i].lat
            diff_long2 = typhoonRecordsList[start - i - 1].long - typhoonRecordsList[start - i].long
            vector2 = [diff_lat2, diff_long2]
            if diff_lat1 == 0 and diff_long1 == 0 or diff_lat2 == 0 and diff_long2 == 0:
                strXLine += ',' + str(0)
            else:
                strXLine += ',' + str(get_vector_angle(vector1, vector2))

        fileXWriter.write(strXLine + '\n')

# Read CMA

In [7]:
import os
import pandas as pd

path = "./data/CMA"
files = os.listdir(path)
files.sort()

pd_list = []
for file in files:
    cma_pd = pd.read_csv(path+'//'+file, delim_whitespace=True, 
                         names=['TROPICALTIME', 'I', 'LAT', 'LONG', 'PRES', 'WND' , 'OWD', 'NAME', 'RECORDTIME'])
    pd_list.append(cma_pd)

df = pd.concat(pd_list, axis=0)

df=df.reset_index(drop=True)

df

,TROPICALTIME,I,LAT,LONG,PRES,WND,OWD,NAME,RECORDTIME
0,66666,0,49,1,0,0,6.0,Carmen,20110729.0
1,1949011300,0,57,1399,1006,0,NaN,NaN,NaN
2,1949011306,0,59,1393,1006,0,NaN,NaN,NaN
3,1949011312,0,63,1387,1006,0,NaN,NaN,NaN
4,1949011318,0,67,1380,1006,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
70651,2018122900,1,115,1244,1000,15,NaN,NaN,NaN
70652,2018122906,1,115,1232,1000,15,NaN,NaN,NaN
70653,2018122912,1,108,1221,1004,13,NaN,NaN,NaN
70654,2018122918,1,100,1210,1004,13,NaN,NaN,NaN


# CMA data

In [8]:

df = df.drop(columns=['OWD','RECORDTIME'])

df = pd.concat([df, pd.DataFrame(columns=['TID','YEAR','MONTH','DAY','HOUR'])], axis=1)

df = df[['TID','YEAR','MONTH','DAY','HOUR','TROPICALTIME', 'I', 'LAT', 'LONG', 'WND', 'PRES', 'NAME']]

tid = 0
name = None
for i in range(0, len(df)):
    if df.at[i, 'TROPICALTIME'] == 66666:
        tid += 1
        name = df.loc[i, 'NAME']
    else:
        df.at[i, 'TID'] = tid
        df.at[i, 'NAME'] = name
        df.at[i, 'YEAR'] = df.loc[i, 'TROPICALTIME'] // 1000000
        df.at[i, 'MONTH'] = df.loc[i, 'TROPICALTIME'] // 10000 % 100
        df.at[i, 'DAY'] = df.loc[i, 'TROPICALTIME'] // 100 % 100
        df.at[i, 'HOUR'] = df.loc[i, 'TROPICALTIME'] % 100

df = df.drop(df[df['TROPICALTIME']==66666].index, axis=0)
df = df.drop(columns=['TROPICALTIME'])

df=df.reset_index(drop=True)

df

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME
0,1,1949,1,13,0,0,57,1399,0,1006,Carmen
1,1,1949,1,13,6,0,59,1393,0,1006,Carmen
2,1,1949,1,13,12,0,63,1387,0,1006,Carmen
3,1,1949,1,13,18,0,67,1380,0,1006,Carmen
4,1,1949,1,14,0,0,72,1373,0,1005,Carmen
...,...,...,...,...,...,...,...,...,...,...,...
68296,2355,2018,12,29,0,1,115,1244,15,1000,(nameless)
68297,2355,2018,12,29,6,1,115,1232,15,1000,(nameless)
68298,2355,2018,12,29,12,1,108,1221,13,1004,(nameless)
68299,2355,2018,12,29,18,1,100,1210,13,1004,(nameless)


In [9]:
df.loc[df['NAME']=='In-fa', 'NAME'] = 'Infa'

In [10]:
df[df['NAME']=='Infa'] 

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME
65420,2260,2015,11,17,0,1,45,1629,13,1002,Infa
65421,2260,2015,11,17,6,1,43,1613,15,1000,Infa
65422,2260,2015,11,17,12,2,46,1597,18,998,Infa
65423,2260,2015,11,17,18,2,52,1583,20,995,Infa
65424,2260,2015,11,18,0,2,55,1571,23,992,Infa
65425,2260,2015,11,18,6,3,58,1557,25,990,Infa
65426,2260,2015,11,18,12,3,63,1547,25,990,Infa
65427,2260,2015,11,18,18,3,71,1539,25,990,Infa
65428,2260,2015,11,19,0,3,79,1529,25,990,Infa
65429,2260,2015,11,19,6,3,85,1516,25,990,Infa


In [11]:
# 增加KEY列
df['KEY'] = None
# 按年份建立记录字典
years = df['YEAR'].unique()
years_dict = dict(zip(years , np.ones(years.shape)))
# 存放计算好的结果
result_list = []
# 不同台风计算当年台风序号
for tid in df['TID'].unique():
    temp_df = df[df['TID']==tid].copy()
    # 跨年台风算前一年的台风
    tid_year = temp_df['YEAR'].unique()[0]
    cy = int(years_dict[tid_year])
    years_dict[tid_year] += 1
    temp_df['KEY'] = str(tid_year) + '-' + str(cy).zfill(2)
    result_list.append(temp_df)
    
df = pd.concat(result_list, axis=0)

# 合并后重置索引
df=df.reset_index(drop=True)

df

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME,KEY
0,1,1949,1,13,0,0,57,1399,0,1006,Carmen,1949-01
1,1,1949,1,13,6,0,59,1393,0,1006,Carmen,1949-01
2,1,1949,1,13,12,0,63,1387,0,1006,Carmen,1949-01
3,1,1949,1,13,18,0,67,1380,0,1006,Carmen,1949-01
4,1,1949,1,14,0,0,72,1373,0,1005,Carmen,1949-01
...,...,...,...,...,...,...,...,...,...,...,...,...
68296,2355,2018,12,29,0,1,115,1244,15,1000,(nameless),2018-33
68297,2355,2018,12,29,6,1,115,1232,15,1000,(nameless),2018-33
68298,2355,2018,12,29,12,1,108,1221,13,1004,(nameless),2018-33
68299,2355,2018,12,29,18,1,100,1210,13,1004,(nameless),2018-33


# Unique

In [12]:
import pandas as pd

df = df.drop(df[~df['HOUR'].isin([0,6,12,18])].index, axis=0)
df=df.reset_index(drop=True)

df

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME,KEY
0,1,1949,1,13,0,0,57,1399,0,1006,Carmen,1949-01
1,1,1949,1,13,6,0,59,1393,0,1006,Carmen,1949-01
2,1,1949,1,13,12,0,63,1387,0,1006,Carmen,1949-01
3,1,1949,1,13,18,0,67,1380,0,1006,Carmen,1949-01
4,1,1949,1,14,0,0,72,1373,0,1005,Carmen,1949-01
...,...,...,...,...,...,...,...,...,...,...,...,...
68083,2355,2018,12,29,0,1,115,1244,15,1000,(nameless),2018-33
68084,2355,2018,12,29,6,1,115,1232,15,1000,(nameless),2018-33
68085,2355,2018,12,29,12,1,108,1221,13,1004,(nameless),2018-33
68086,2355,2018,12,29,18,1,100,1210,13,1004,(nameless),2018-33


In [13]:
df = df.drop_duplicates()
df=df.reset_index(drop=True)

df

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME,KEY
0,1,1949,1,13,0,0,57,1399,0,1006,Carmen,1949-01
1,1,1949,1,13,6,0,59,1393,0,1006,Carmen,1949-01
2,1,1949,1,13,12,0,63,1387,0,1006,Carmen,1949-01
3,1,1949,1,13,18,0,67,1380,0,1006,Carmen,1949-01
4,1,1949,1,14,0,0,72,1373,0,1005,Carmen,1949-01
...,...,...,...,...,...,...,...,...,...,...,...,...
68083,2355,2018,12,29,0,1,115,1244,15,1000,(nameless),2018-33
68084,2355,2018,12,29,6,1,115,1232,15,1000,(nameless),2018-33
68085,2355,2018,12,29,12,1,108,1221,13,1004,(nameless),2018-33
68086,2355,2018,12,29,18,1,100,1210,13,1004,(nameless),2018-33


In [14]:

df.to_csv('./data/raw.csv', index=False)

print('raw.csv 保存成功')

raw.csv 保存成功


# Data Preprocessing

In [15]:
import pandas as pd

# 每一个台风划分为一个dataframe
df = pd.read_csv('./data/raw.csv')

tids = df['TID'].unique()

cma_list = []

for tid in tids:
    temp_df = df[df['TID']==tid]
    temp_df = temp_df.reset_index(drop=True)
    cma_list.append(temp_df)
    
len(cma_list)

2355

In [17]:
valid_tropical_len = forward_seq + valid_tropical_seq_num + backward_seq

temp = []
for df in cma_list:
    if df.shape[0] >= valid_tropical_len:
        temp.append(df)

cma_list = temp

len(cma_list)

2118

In [18]:
df = pd.concat(cma_list, axis=0)
df=df.reset_index(drop=True)
train_range = [ str(x) for x in range(trainYear[0], trainYear[1]+1) ]
train_keys =  [v for i, v in enumerate(df['KEY'].unique()) if any(s in v for s in train_range)]

test_range = [ str(x) for x in range(testYear[0], testYear[1]+1) ]
test_keys =  [v for i, v in enumerate(df['KEY'].unique()) if any(s in v for s in test_range)]

df = df[(df['KEY'].isin(train_keys)) | (df['KEY'].isin(test_keys))]
df=df.reset_index(drop=True)

In [19]:
tname = pd.read_csv('./data/typhoon_name.csv')

In [20]:
dict_name = {}
for i in range(len(tname)):
    dict_name[tname.at[i, 'en'].lower()] = tname.at[i, 'cn']
dict_name['(nameless)']='未命名'

In [21]:
df['CN_NAME'] = None
for i in range(len(df)):
    try:
        df.at[i, 'CN_NAME'] = dict_name[df.at[i, 'NAME'].lower()]
    except KeyError:
        print(df.at[i, 'NAME'].lower())

In [22]:
df.to_csv('./data/pre_processing.csv', index=False)

df

,TID,YEAR,MONTH,DAY,HOUR,I,LAT,LONG,WND,PRES,NAME,KEY,CN_NAME
0,1827,2000,5,5,0,1,96,1360,15,1004,Damrey,2000-01,达维
1,1827,2000,5,5,6,1,99,1350,15,1004,Damrey,2000-01,达维
2,1827,2000,5,5,12,1,102,1344,15,1004,Damrey,2000-01,达维
3,1827,2000,5,5,18,1,107,1337,15,1002,Damrey,2000-01,达维
4,1827,2000,5,6,0,1,111,1330,15,1002,Damrey,2000-01,达维
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2355,2018,12,29,0,1,115,1244,15,1000,(nameless),2018-33,未命名
14989,2355,2018,12,29,6,1,115,1232,15,1000,(nameless),2018-33,未命名
14990,2355,2018,12,29,12,1,108,1221,13,1004,(nameless),2018-33,未命名
14991,2355,2018,12,29,18,1,100,1210,13,1004,(nameless),2018-33,未命名


In [23]:
typhoonHeaderList = read_cma_dfs(cma_list)

for i in range(1, predict_seq_num+1):
    trainXFile = open('./data/CMA_train_'+str(i*6)+'h.csv', 'w')
    testXFile = open('./data/CMA_test_'+str(i*6)+'h.csv', 'w')
    for typhoonHeader in typhoonHeaderList:
        typhoonRecordsList = typhoonHeader.typhoonRecords
        if typhoonHeader.typhoonYear in range(trainYear[0], trainYear[1]+1):
            buildup_feature(typhoonRecordsList, typhoonHeader.tid, trainXFile, i)
        elif typhoonHeader.typhoonYear in range(testYear[0], testYear[1]+1):
            buildup_feature(typhoonRecordsList, typhoonHeader.tid, testXFile, i)

    trainXFile.close()
    testXFile.close()

# For Reanalysis data

In [24]:
cma_ecwmf_train = open('./data/cma_ecwmf_train.csv', 'w')
cma_ecwmf_test = open('./data/cma_ecwmf_test.csv', 'w')

for typhoonHeader in typhoonHeaderList:
    typhoonRecordsList = typhoonHeader.typhoonRecords
    for start in range(0, len(typhoonRecordsList) - 4):
        strXLine = str(typhoonRecordsList[start].totalNum) + \
           ',' + str(typhoonRecordsList[start].typhoonTime) + \
           ',' + str(typhoonRecordsList[start].lat) + \
           ',' + str(typhoonRecordsList[start].long) + \
           ',' + str(typhoonHeader.tid)
        if typhoonHeader.typhoonYear in range(trainYear[0], trainYear[1]+1):
            cma_ecwmf_train.write(strXLine + '\n')
        elif typhoonHeader.typhoonYear in range(testYear[0], testYear[1]+1):
            cma_ecwmf_test.write(strXLine + '\n')

cma_ecwmf_train.close()
cma_ecwmf_test.close()